In [1]:
from owlready2 import *
import rdflib
import rdflib.plugins.sparql as sparql

In [32]:
onto = get_ontology("data/wildlife-en.xml").load()
# default_world.graph.dump()

In [40]:
graph = default_world.as_rdflib_graph()
query = sparql.prepareQuery("""
                            PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
                            PREFIX me: <file:wildlife-en.xml#>
                            SELECT ?b WHERE {
                            ?b rdf:type me:lion .
                            }
""")
qres = graph.query(query)

In [35]:
print(qres)
for v in qres:
    print(v)

(rdflib.term.URIRef('file:wildlife.owl#Léo'),)


In [36]:
def into_owlready(x):
    global onto
    if (str(type(x))=="<class 'rdflib.term.URIRef'>"):
        r = onto.search(iri=str(x))
        if r:
            return r
        else:
            return None
    else:
        return None

In [45]:
into_owlready(rdflib.term.URIRef('file:wildlife.owl#animal'))

[wildlife-new.animal]

In [44]:
onto = get_ontology("data/wildlife-new.owl").load()
#sync_reasoner([onto])
graph = default_world.as_rdflib_graph()
q = sparql.prepareQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX me: <file:wildlife-en.xml#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
SELECT ?b ?x WHERE { 
?b rdf:type owl:Class .
FILTER ( regex(str(?b),".*plant") )
}""")
qres = graph.query(q)
    
for v in qres:
    print(v)

(rdflib.term.URIRef('file:wildlife.owl#plant'), None)
(rdflib.term.URIRef('file:wildlife.owl#edible-plant'), None)
(rdflib.term.URIRef('file:wildlife.owl#plant'), None)
(rdflib.term.URIRef('file:wildlife.owl#edible-plant'), None)


In [48]:
onto = get_ontology("data/wildlife-new.owl").load()
#sync_reasoner([onto])
graph = default_world.as_rdflib_graph()
q = sparql.prepareQuery(
    """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX me: <file:wildlife.owl#>
    SELECT ?b WHERE {
    ?x rdfs:subClassOf* me:animal .
    ?b rdf:type ?x .
    ?b rdf:type owl:NamedIndividual .
    }
    """
)
qres = graph.query(q)
    
for v in qres:
    print(v)

(rdflib.term.URIRef('file:wildlife.owl#Gigi'),)
(rdflib.term.URIRef('file:wildlife.owl#Gigi'),)
(rdflib.term.URIRef('file:wildlife.owl#Giginou'),)
(rdflib.term.URIRef('file:wildlife.owl#Giginou'),)
(rdflib.term.URIRef('file:wildlife.owl#Léo'),)
(rdflib.term.URIRef('file:wildlife.owl#Léo'),)
(rdflib.term.URIRef('file:wildlife.owl#Gigi'),)
(rdflib.term.URIRef('file:wildlife.owl#Gigi'),)
(rdflib.term.URIRef('file:wildlife.owl#Giginou'),)
(rdflib.term.URIRef('file:wildlife.owl#Giginou'),)
(rdflib.term.URIRef('file:wildlife.owl#Léo'),)
(rdflib.term.URIRef('file:wildlife.owl#Léo'),)


In [60]:
q = sparql.prepareQuery(
    """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX me: <file:wildlife.owl#>
    SELECT ?b WHERE {
    ?x rdfs:subClassOf* me:animals .
    ?b rdf:type ?x .
    ?b rdf:type owl:NamedIndividual .
    }
    """
)
qres = graph.query(q)
    
for v in qres:
    print(v)

In [2]:
def into_owlready(x):
    global onto
    if (str(type(x))=="<class 'rdflib.term.URIRef'>"):
        r = onto.search(iri=str(x))
        if r:
            return r
        else:
            return None
    else:
        return None

In [3]:
def build_SPARQL_query(classe):
    return """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX me: <file:wildlife.owl#>
SELECT ?b WHERE {
?x rdfs:subClassOf* me:%s .
?b rdf:type ?x .
?b rdf:type owl:NamedIndividual .
}""" % classe

In [5]:
onto = get_ontology("data/wildlife-new.owl").load()
graph = default_world.as_rdflib_graph()

In [31]:
def what_are_the_(classeName):
    query = sparql.prepareQuery(
        build_SPARQL_query(classeName[:-1])
    )
    response = graph.query(query)
    response = [v for v in response]
    if len(response) == 0:
        return "There is no {}.".format(classeName[:-1])
    elif len(response) == 1:
        name = str(into_owlready(response[0][0])[0]).split('.')[-1]
        return "There is a single {}. It is {}.".format(classeName[:-1], name)
    else:
        names = [str(into_owlready(resp[0])[0]).split('.')[-1] for resp in response]
        return "There are {} {}. They are {}.".format(len(response), classeName, ", ".join(names))
        


In [32]:
print(what_are_the_("animals"))
print(what_are_the_("lions"))

There are 3 animals. They are Gigi, Giginou, Léo.
There is a single lion. It is Léo.


In [33]:
def is_a(name, classeName):
    query = sparql.prepareQuery(
        build_SPARQL_query(classeName)
    )
    response = graph.query(query)
    response = [v for v in response]
    if len(response) > 0:
        names = [str(into_owlready(resp[0])[0]).split('.')[-1] for resp in response]
        if name in names:
            return "Yes."        
    return "No."  
    

In [34]:
print(is_a('Léo', 'lion'))
print(is_a('Gigi', 'lion'))

Yes.
No.


In [ ]:
def what_do_we_know_about(name):
    
    

    return "{} is a {}. It weights {}kg and its size is {}m.".format(name, className, weight, size)

(As well as other pieces of information such that “It eats XXX”